# Topic modeling analysis

#### Description
- All 11 qualitative questions analyzed for key topics
- Number of topics varied within the recommended range of 2 - 5 based on unique key words for each group. This was a subjective process that had to be completed by adjusting the number of components, examing the key words, looking at the intertopic distance in the visualizations and reading the most relevant responses to each topic.
- Non-cleaned text of must prominent topcs was included to make reading easier.

### Import libraries and data

In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../')


import nlp
import wrangle

import nltk

from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
datadf, dictionarydf = wrangle.wrangle_data(path_prefix='../')

### Viewing the imported data

In [149]:
datadf.head(1)

,job_title,job_taxo,job_id,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,num_employees,num_researchers,primary_industry,types_res_used,future_res,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,research_educ,research_educ_cat,research_educ_desc,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,how_pick_events,best_event,events_attend_recent,ideal_conference_size,ideal_conference_size_cat,ideal_structure,ideal_structure_cat,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables,other_conference_types,ideal_topics,ideal_attendees,recommendations,persona_id
resp_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
284,Academic,4,1.0,3,2,0,3,3,3,3,2,1,4,0.0,2.0,maritime,both qualitative and quantitative research,both qualitative and quantitative research,5,5,5,5,5,5,5,5,5,5,"sort of, psychology background.",True,NaN,0,1,1,2,3,1,3,3,2,3,3,4,4,3,3,4,2,3,1,3,1,3,4,NaN,NaN,chi interact,depending on the professional values,7.0,"any, depending on the values",5.0,False,False,False,4,2,2,2,3,2,2,2,NaN,researchpractice connection,NaN,research-practice connections,2


In [5]:
datadf.describe()

,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,2.856749,2.913223,0.402204,2.600551,1.982094,2.931129,1.845730,2.924242,0.776860,1.860882,3.283747,3.323691,0.960055,2.633609,3.084022,3.139118,1.862259,2.906336,1.326446,1.632231,2.074380,2.659780,1.801653,1.451791,1.663912,2.982094,3.500000,3.263085,2.079890,3.053719,2.202479,2.250689,2.360882,2.236915,2.871901,3.652893,2.776860,2.209366,1.998623,2.028926,1.903581,2.944904,3.336088,2.706612,2.754821,2.341598,2.988981,2.173554,1.480716,2.121212,2.431129
std,1.025941,0.939210,0.720686,1.166285,1.134273,0.950177,1.236637,1.056235,0.914143,1.486931,1.571974,1.553165,1.608530,1.702955,1.694565,1.589165,1.684913,1.711475,1.654099,1.674476,0.791318,0.571707,0.836609,0.784165,0.761119,0.960436,0.711967,0.834391,1.213355,0.983242,1.150254,1.143723,1.101378,1.157435,1.071954,0.590183,0.926135,1.168658,1.107341,1.158115,1.263411,1.019666,0.971309,1.124344,0.977229,1.106273,1.109154,1.219458,1.239149,1.179908,1.211472
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,3.000000,0.000000,0.000000,2.000000,2.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,1.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,2.000000
50%,3.000000,3.000000,0.000000,3.000000,2.000000,3.000000,2.000000,3.000000,1.000000,2.000000,4.000000,4.000000,0.000000,2.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,3.000000,2.000000,1.000000,2.000000,3.000000,4.000000,3.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,2.000000,3.000000
75%,4.000000,3.000000,1.000000,3.000000,3.000000,4.000000,3.000000,4.000000,1.000000,3.000000,5.000000,5.000000,1.000000,4.000000,5.000000,5.000000,3.000000,5.000000,2.000000,3.000000,3.000000,3.000000,2.000000,2.000000,2.000000,4.000000,4.000000,4.0000

In [6]:
datadf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 284 to 2
Data columns (total 77 columns):
job_title                    726 non-null category
job_taxo                     726 non-null category
job_id                       723 non-null category
job_conduct_res              726 non-null int64
job_analyze_res              726 non-null int64
job_buy_res_report           726 non-null int64
job_manage_res_proj          726 non-null int64
job_observe_res              726 non-null int64
job_plan_res                 726 non-null int64
job_teach_res                726 non-null int64
job_advocate_res             726 non-null int64
job_hire_res_vendor          726 non-null int64
job_lead_res_team            726 non-null int64
num_employees                723 non-null category
num_researchers              722 non-null category
primary_industry             726 non-null category
types_res_used               690 non-null category
future_res                   609 non-null category
exp_con

# Analysis of qualitative questions

## 5. What is your company or organization's primary industry?

#### Clean and lemmatize the data for this question

In [7]:
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.basic_clean)
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.lemmatize)

#### Create word count matrix and vector. Settings: 2 word ngrams permitted; words in more than 30% of documents were ignored.

In [150]:
primary_industry_matrix, primary_industry_vector = nlp.create_wordcount_matrix(datadf.primary_industry.dropna(), ngram=(1,3), max_df=.3)
primary_industry_matrix, primary_industry_vector

(<726x180 sparse matrix of type '<class 'numpy.int64'>'
 	with 1239 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

#### Apply LDA method using 4, 6 and 8 components (can be changed) and a random state set to ensure the results can be replicated.

In [214]:
lda5 = LatentDirichletAllocation(n_components= 6, random_state = 42)

In [215]:
lda5.fit(primary_industry_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=6, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [216]:
pyLDAvis.sklearn.prepare(lda5, primary_industry_matrix, primary_industry_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.242712 -0.095023       1        1  18.949498
1      0.191157  0.137655       2        1  18.266455
5     -0.180426  0.116459       3        1  17.709263
3     -0.114651 -0.241337       4        1  17.006357
4     -0.085096  0.174884       5        1  14.685549
0     -0.053696 -0.092637       6        1  13.382878, topic_info=    Category       Freq                    Term      Total  loglift  logprob
151  Default  38.000000                software  38.000000  30.0000  30.0000
159  Default  39.000000              technology  39.000000  29.0000  29.0000
75   Default  46.000000              healthcare  46.000000  28.0000  28.0000
67   Default  37.000000                 fintech  37.000000  27.0000  27.0000
157  Default  38.000000                    tech  38.000000  26.0000  26.0000
51   Default  37.000000               education  37.000000  25.0000  25.0000
26   Default  33.000000              consulting  33.000000  24.0000  24.0000
65   Default  25.000000               financial  25.000000  23.0000  23.0000
142  Default  23.000000                  retail  23.000000  22.0000  22.0000
71   Default  30.000000              government  30.000000  21.0000  21.0000
148  Default  38.000000                 service  38.000000  20.0000  20.0000
66   Default  19.000000       financial service  19.000000  19.0000  19.0000
23   Default  14.000000             consultancy  14.000000  18.0000  18.0000
143  Default  15.000000                    saas  15.000000  17.0000  17.0000
86   Default  21.000000                industry  21.000000  16.0000  16.0000
47   Default  16.000000               ecommerce  16.000000  15.0000  15.0000
136  Default  22.000000                research  22.000000  14.0000  14.0000
39   Default  31.000000                  design  31.000000  13.0000  13.0000
92   Default  15.000000               insurance  15.000000  12.0000  12.0000
63   Default  13.000000                 finance  13.000000  11.0000  11.0000
128  Default  10.000000                  public  10.000000  10.0000  10.0000
55   Default   9.000000              enterprise   9.000000   9.0000   9.0000
7    Default   9.000000                 banking   9.000000   8.0000   8.0000
163  Default   8.000000          transportation   8.000000   7.0000   7.0000
146  Default   9.000000                  sector   9.000000   6.0000   6.0000
74   Default   7.000000                  health   7.000000   5.0000   5.0000
107  Default   7.000000               marketing   7.000000   4.0000   4.0000
118  Default   9.000000               nonprofit   9.000000   3.0000   3.0000
111  Default  14.000000                  medium  14.000000   2.0000   2.0000
61   Default   8.000000                 fashion   8.000000   1.0000   1.0000
..       ...        ...                     ...        ...      ...      ...
160   Topic6   5.155081                 telecom   5.870981   1.8812  -3.4773
129   Topic6   5.154688           public sector   5.870921   1.8811  -3.4774
81    Topic6   4.157139                      hr   4.873198   1.8523  -3.6925
146   Topic6   7.952776                  sector   9.675405   1.8151  -3.0438
29    Topic6   3.159571    consumer electronics   3.875462   1.8070  -3.9669
52    Topic6   3.159571             electronics   3.875462   1.8070  -3.9669
82    Topic6   3.159311                 hr tech   3.875429   1.8069  -3.9669
69    Topic6   3.159142               freelance   3.875385   1.8068  -3.9670
24    Topic6   2.161811  consultancy government   2.877701   1.7251  -4.3464
130   Topic6   2.161810          public service   2.877701   1.7251  -4.3464
46    Topic6   2.161810                  domain   2.877701   1.7251  -4.3464
156   Topic6   2.161528                  supply   2.877666   1.7250  -4.3465
126   Topic6   2.161457    professional service   2.877638   1.7250  -4.3465
125   Topic6   2.161457            professional   2.877638  

#### Topic groups (4)
1. Education 
2. Fintech 
3. Healthcare 
4. Tech 

#### Topic groups (6) 
1. fintech
2. service
3. healthcare
4. technology
5. software
6. consultancy

#### Topic groups (8) - 5 & 8 overlap
1. fintech
2. education
3. technology
4. tech
5. software
6. healthcare
7. government
8. public


In [12]:
lda_W5 = lda5.transform(primary_industry_matrix)

In [13]:
top_doc_column5 = datadf.primary_industry.dropna()

#### Create second word count matrix. Excludes words in appear in 80%+ of all documents and in 2 or fewer documents. Include bigrams.

In [14]:
word_count_matrix5, count_vect5 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [15]:
LDA5a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA5a.fit(word_count_matrix5)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [16]:
lda_H = LDA5a.transform(word_count_matrix5)

In [17]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column5, 3)

Top 3 Documents for Topic 0: 

Document 1
real estate tech

Document 2
real estate tech

Document 3
sotware development across domain such a data science deep learning health tech interaction etc

Top 3 Documents for Topic 1: 

Document 1
design research service and innovation planning aka design planning design ledstrategy

Document 2
market research agency 95 client are in financial service

Document 3
ux research we're a ux research agency a vendor used by fortune 500 company

Top 3 Documents for Topic 2: 

Document 1
commercial real estate

Document 2
commercial real estate

Document 3
architecture engineering construction aec industry manufacturing design medium entertainment industry

Top 3 Documents for Topic 3: 

Document 1
banking retail edu logistic government entertainment tourism aviation

Document 2
financial sector public sector resource technology retail healthcare

Document 3
aerospaceautomativebankingcentral government chemical consumer package goodsdefenceeducation ma

## 6. What types of research do you currently use to make decisions?

In [18]:
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.basic_clean)
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.lemmatize)

In [231]:
types_res_used_matrix, types_res_used_vector = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), ngram=(1,3), max_df=.3)
types_res_used_matrix, types_res_used_vector

(<690x1209 sparse matrix of type '<class 'numpy.int64'>'
 	with 7564 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [247]:
lda6 = LatentDirichletAllocation(n_components= 7, random_state = 42)

In [248]:
lda6.fit(types_res_used_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [249]:
pyLDAvis.sklearn.prepare(lda6, types_res_used_matrix, types_res_used_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.071478  0.059933       1        1  20.727776
6     -0.085695 -0.073155       2        1  18.723355
0     -0.123895  0.033381       3        1  15.761362
2      0.175815 -0.048143       4        1  13.449097
3      0.100097  0.169731       5        1  12.685934
1     -0.039804  0.000024       6        1  10.373322
5      0.044959 -0.141770       7        1   8.279154, topic_info=     Category        Freq                         Term       Total  loglift  \
659   Default  143.000000                  qualitative  143.000000  30.0000   
693   Default   74.000000                 quantitative   74.000000  29.0000   
501   Default   34.000000                       market   34.000000  28.0000   
502   Default   27.000000              market research   27.000000  27.0000   
123   Default   44.000000           contextual inquiry   44.000000  26.0000   
369   Default   45.000000                      inquiry   45.000000  25.0000   
442   Default   35.000000          interview usability   35.000000  24.0000   
279   Default   43.000000                  focus group   43.000000  23.0000   
308   Default   47.000000                        group   47.000000  22.0000   
278   Default   47.000000                        focus   47.000000  21.0000   
834   Default   36.000000                         sort   36.000000  20.0000   
236   Default   39.000000                   evaluative   39.000000  19.0000   
68    Default   35.000000                    card sort   35.000000  18.0000   
673   Default   26.000000     qualitative quantitative   26.000000  17.0000   
445   Default   19.000000  interview usability testing   19.000000  16.0000   
1173  Default   27.000000                           ux   27.000000  15.0000   
296   Default   40.000000                   generative   40.000000  14.0000   
530   Default   17.000000                        mixed   17.000000  13.0000   
297   Default   19.000000        generative evaluative   19.000000  12.0000   
163   Default   47.000000                       design   47.000000  11.0000   
147   Default   59.000000                         data   59.000000  10.0000   
1112  Default  147.000000            usability testing  147.000000   9.0000   
656   Default   41.000000                         qual   41.000000   8.0000   
28    Default   78.000000                    analytics   78.000000   7.0000   
120   Default   76.000000                   contextual   76.000000   6.0000   
657   Default   20.000000                   qual quant   20.000000   5.0000   
510   Default   51.000000                       method   51.000000   4.0000   
804   Default   21.000000                    secondary   21.000000   3.0000   
67    Default   61.000000                         card   61.000000   2.0000   
864   Default  112.000000                        study  112.000000   1.0000   
...       ...         ...                          ...         ...      ...   
747    Topic7    4.129742   research indepth interview    5.854885   2.1424   
283    Topic7    2.880601        focus group usability    4.219397   2.1097   
314    Topic7    2.880601              group usability    4.219397   2.1097   
327    Topic7    4.224711                         help    6.750920   2.0227   
1173   Topic7   13.006110                           ux   27.041716   1.7595   
746    Topic7    3.997751             research indepth    6.699138   1.9752   
1174   Topic7    6.584563                  ux research   12.619101   1.8409   
898    Topic7    2.607991                         sure    4.199753   2.0150   
346    Topic7    2.609858                         idis    4.225553   2.0096   
279    Topic7   13.165833                  focus group   43.653730   1.2928   
278    Topic7   13.612457                        focus   47.975671   1.2317   
163    Topic7   12.830622                       design   47.181822   1.189

#### Topic groups (3)
1. usability testing
2. qualitative
3. study

#### Topic groups (4)
1. usability testing
2. study
3. user 
4. testing

#### Topic groups (6) 
1. user
2. card
3. usability testing
4. qualitative
5. focus group
6. generative evaluation

#### Topic groups (8) - 8 overlaps with 2 & 6
1. user
2. contextual inquiry
3. usability testing
4. qualitative
5. market research
6. usability testing, alternate: interview survey
7. indepth interview
8. usability testing

In [23]:
lda_W6 = lda6.transform(types_res_used_matrix)

In [24]:
top_doc_column6 = datadf.types_res_used.dropna()

In [25]:
word_count_matrix6, count_vect6 = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [26]:
LDA6a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA6a.fit(word_count_matrix6)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [27]:
lda_H = LDA6a.transform(word_count_matrix6)

In [28]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column6, 3)

Top 3 Documents for Topic 0: 

Document 1
qualitative interview survey card sort prototype testing usability testing analytics etc

Document 2
usability testing moderated and unmoderated card sort tree jacking max diff jtbd odi survey observational ethnographic research concept validation others a needed

Document 3
usability testing empathy1x1 interview survey card sort tree test prototype and concept testing diary study eye tracking

Top 3 Documents for Topic 1: 

Document 1
user research participant observation interview contextual enquiry usability testing feedback analysis data analysis survey

Document 2
contextual inquiry interview survey feedback form quant behavior analysis quant usability study usability testing ux workshop

Document 3
foundational product research usability testing 11 interview unmoderated user testing using usertestingcom survey contextual inquiry diary study analytics

Top 3 Documents for Topic 2: 

Document 1
qual research interview in lab moderated usabi

## 7. What types of research are you considering in the future?

In [29]:
datadf.future_res = datadf.future_res.dropna().apply(nlp.basic_clean)
datadf.future_res = datadf.future_res.dropna().apply(nlp.lemmatize)

In [227]:
future_res_matrix, future_res_vector = nlp.create_wordcount_matrix(datadf.future_res.dropna(), ngram=(1,3), max_df=.3)
future_res_matrix, future_res_vector

(<609x540 sparse matrix of type '<class 'numpy.int64'>'
 	with 3008 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [228]:
lda7 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [229]:
lda7.fit(future_res_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [230]:
pyLDAvis.sklearn.prepare(lda7, future_res_matrix, future_res_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.011066 -0.162547       1        1  30.652362
3      0.020336 -0.008149       2        1  26.605634
1     -0.188220  0.087641       3        1  23.924497
2      0.178950  0.083055       4        1  18.817507, topic_info=    Category        Freq                   Term       Total  loglift  logprob
371  Default  138.000000               research  138.000000  30.0000  30.0000
434  Default   69.000000                  study   69.000000  29.0000  29.0000
90   Default   39.000000                  diary   39.000000  28.0000  28.0000
459  Default   91.000000                testing   91.000000  27.0000  27.0000
91   Default   35.000000            diary study   35.000000  26.0000  26.0000
343  Default   36.000000           quantitative   36.000000  25.0000  25.0000
76   Default   43.000000                   data   43.000000  24.0000  24.0000
0    Default   20.000000                     ab   20.000000  23.0000  23.0000
506  Default   32.000000      usability testing   32.000000  22.0000  22.0000
2    Default   19.000000             ab testing   19.000000  21.0000  21.0000
12   Default   31.000000              analytics   31.000000  20.0000  20.0000
85   Default   28.000000                 design   28.000000  19.0000  19.0000
360  Default   29.000000                 remote   29.000000  18.0000  18.0000
330  Default   26.000000            qualitative   26.000000  17.0000  17.0000
275  Default   17.000000                    new   17.000000  16.0000  16.0000
481  Default   11.000000               tracking   11.000000  15.0000  15.0000
299  Default    8.000000          participatory    8.000000  14.0000  14.0000
443  Default    8.000000                   sure    8.000000  13.0000  13.0000
61   Default   22.000000             contextual   22.000000  12.0000  12.0000
522  Default    9.000000                  using    9.000000  11.0000  11.0000
501  Default   53.000000              usability   53.000000  10.0000  10.0000
154  Default    7.000000             generative    7.000000   9.0000   9.0000
10   Default   22.000000               analysis   22.000000   8.0000   8.0000
25   Default   10.000000           benchmarking   10.000000   7.0000   7.0000
491  Default   13.000000                   type   13.000000   6.0000   6.0000
168  Default   10.000000                   help   10.000000   5.0000   5.0000
347  Default    9.000000  quantitative research    9.000000   4.0000   4.0000
407  Default    9.000000                session    9.000000   3.0000   3.0000
494  Default   29.000000            unmoderated   29.000000   2.0000   2.0000
444  Default   37.000000                 survey   37.000000   1.0000   1.0000
..       ...         ...                    ...         ...      ...      ...
338   Topic4    2.710536             quant data    3.353705   1.4575  -5.3781
161   Topic4    2.710314                   good    3.353672   1.4574  -5.3781
410   Topic4    2.709932                similar    3.353676   1.4572  -5.3783
169   Topic4    2.708774            help inform    3.353787   1.4568  -5.3787
142   Topic4    2.707678              formative    3.353740   1.4564  -5.3791
432   Topic4    2.707348               stronger    3.353768   1.4563  -5.3792
376   Topic4    2.691751        research design    3.353912   1.4504  -5.3850
396   Topic4    5.903591                science    7.546364   1.4249  -4.5996
147   Topic4    4.398468                 future    5.874220   1.3811  -4.8939
190   Topic4    1.877268                   info    2.519299   1.3762  -5.7454
379   Topic4    1.877255       research finding    2.519299   1.3762  -5.7454
397   Topic4    1.877255              scientist    2.519299   1.3762  -5.7454
80    Topic4    1.877255         data scientist    2.519299   1.3762  -5.7454
168   Topic4    7.620338                   help   10.943948   1.3084  -4.3444
380   Topic4    3.552830        research meth

#### Topic groups (4)

1. testing
2. quantitative
3. study
4. data

#### Topic groups (6) - 4 & 6 overlap
1. quantitative
2. usability testing
3. study
4. analytics
5. diary study
6. sure


#### Topic groups (8) - 5 is almost completely within 2; 6 & 8 overlap
1. quantitative
2. testing
3. diary study
4. data
5. testing
6. focus group
7. ethnographic
8. analytics


In [34]:
lda_W7 = lda7.transform(future_res_matrix)

In [35]:
top_doc_column7 = datadf.future_res.dropna()

In [36]:
word_count_matrix7, count_vect7 = nlp.create_wordcount_matrix(datadf.future_res.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [37]:
LDA7a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA7a.fit(word_count_matrix7)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [38]:
lda_H = LDA7a.transform(word_count_matrix7)

In [39]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column7, 3)


Top 3 Documents for Topic 0: 

Document 1
user experience research workshop survey interview usability testing benchmark card sorting etc and i would like to delve deeper in analytics

Document 2
doing more strategic piece to help inform the product strategy early on before the okrs are set have fundamental understanding our user in different market since we have a global product and we havent done a lot of thinking around localized content and market difference doing information architecture research

Document 3
abn multivariate testing analytic research behavioural research card sortingtree testing content auditing experience mapping information architecture ia lab testing build persona development research participant recruitment management scenariotask mapping survey design analysis usability testing userstakeholder interview workshop

Top 3 Documents for Topic 1: 

Document 1
much more usability testing heatmappingbehavior tracking along the line of what pendoio offer exploratory 

## 10. Describe your educational background with research

In [40]:
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.basic_clean)
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.lemmatize)

In [223]:
research_educ_desc_matrix, research_educ_desc_vector = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), ngram=(1,3), max_df=.3)
research_educ_desc_matrix, research_educ_desc_vector

(<540x1118 sparse matrix of type '<class 'numpy.int64'>'
 	with 6604 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [224]:
lda10 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [225]:
lda10.fit(research_educ_desc_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [226]:
pyLDAvis.sklearn.prepare(lda10, research_educ_desc_matrix, research_educ_desc_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.034791 -0.118467       1        1  42.405557
2     -0.101616  0.085207       2        1  35.110446
0      0.136407  0.033260       3        1  22.483997, topic_info=     Category        Freq                       Term       Total  loglift  \
235   Default   99.000000                     degree   99.000000  30.0000   
258   Default  160.000000                     design  160.000000  29.0000   
590   Default  128.000000                     master  128.000000  28.0000   
692   Default   74.000000                        phd   74.000000  27.0000   
594   Default   39.000000              master degree   39.000000  26.0000   
1051  Default   66.000000                         ux   66.000000  25.0000   
402   Default   29.000000                       grad   29.000000  24.0000   
493   Default   29.000000                interaction   29.000000  23.0000   
403   Default   24.000000                grad school   24.000000  22.0000   
36    Default   33.000000               anthropology   33.000000  21.0000   
422   Default   39.000000                        hci   39.000000  20.0000   
903   Default   22.000000                  sociology   22.000000  19.0000   
89    Default   24.000000                       book   24.000000  18.0000   
144   Default   21.000000                  cognitive   21.000000  17.0000   
866   Default   44.000000                     school   44.000000  16.0000   
285   Default   24.000000            design research   24.000000  15.0000   
349   Default   19.000000               experimental   19.000000  14.0000   
167   Default   12.000000                   computer   12.000000  13.0000   
1043  Default   22.000000                       user   22.000000  12.0000   
1063  Default   21.000000                ux research   21.000000  11.0000   
651   Default   13.000000                        msc   13.000000  10.0000   
896   Default   27.000000                     social   27.000000   9.0000   
344   Default   21.000000                 experience   21.000000   8.0000   
454   Default   10.000000  humancomputer interaction   10.000000   7.0000   
453   Default   10.000000              humancomputer   10.000000   6.0000   
870   Default   45.000000                    science   45.000000   5.0000   
71    Default   18.000000                   bachelor   18.000000   4.0000   
373   Default   19.000000                      focus   19.000000   3.0000   
603   Default    9.000000               master human    9.000000   2.0000   
784   Default   15.000000                       read   15.000000   1.0000   
...       ...         ...                        ...         ...      ...   
985    Topic3    3.513276                     theory    4.116000   1.3340   
606    Topic3    4.209770        master human factor    4.939390   1.3325   
597    Topic3    3.508280        master degree human    4.116392   1.3325   
417    Topic3    3.507412                       half    4.116404   1.3323   
693    Topic3    3.499552           phd anthropology    4.117106   1.3299   
22     Topic3    3.494619                advertising    4.117640   1.3283   
309    Topic3    3.465472                    diploma    4.119904   1.3194   
519    Topic3    3.462241                       just    4.120180   1.3184   
594    Topic3   27.436321              master degree   39.099056   1.1381   
235    Topic3   63.257099                     degree   99.522793   1.0392   
903    Topic3   16.248644                  sociology   22.460345   1.1686   
493    Topic3   19.789294                interaction   29.220087   1.1027   
36     Topic3   21.648695               anthropology   33.421993   1.0581   
590    Topic3   66.236799                     master  128.268727   0.8315   
422    Topic3   22.864925                        hci   39.496945   0.9457   
692    Topic3   33.305835                        phd   74.527948   0

#### Topic groups (2) - no overlapping circles
1. ux
2. market

#### Topic groups (3) - no overlapping circles
1. design class
2. design course
3. master degree

#### Topic groups (4) - 1 & 3 overlap
1. design
2. class
3. design
4. master degree

#### Topic groups (6) - 1 &4, 2 & 3 overlap
1. design
2. class
3. phd
4. design
5. psychology
6. master degree

#### Topic groups (8) - 1 & 5, 3 & 4 overlap
1. design
2. class
3. method
4. phd
5. design
6. class
7. master degree
8. hci

In [45]:
lda_W10 = lda10.transform(research_educ_desc_matrix)

In [46]:
top_doc_column10 = datadf.research_educ_desc.dropna()

In [47]:
word_count_matrix10, count_vect10 = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [48]:
LDA10a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA10a.fit(word_count_matrix10)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [49]:
lda_H = LDA10a.transform(word_count_matrix10)

In [50]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column10, 3)

Top 3 Documents for Topic 0: 

Document 1
i took professional development course in research up to one year in length at undergraduate level and then started postgraduate level private tuition i'd love to do a master or phd degree but am not able to manage the commitment even parttime alongside working full time i continue to take short course to increase my knowledge and develop my skill

Document 2
phd in human computer interaction research method data analysis coursework in various area eg ethnography statistic a well a supporting other people's research and eventually running my own research with supervision and advice from more senior people

Document 3
undergraduate degree in psychology extensive study of behavioral and scientific research methodology and analysis qual and quant interned and worked in lab for several year genetics neuroethology biopsychology graduate degree in architecture extensive study and practice of many type of research method

Top 3 Documents for Topic 1: 

## 14. How do you decide which events to attend?

In [51]:
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.basic_clean)
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.lemmatize)

In [52]:
how_pick_events_matrix, how_pick_events_vector = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), ngram=(1,3), max_df=.4)
how_pick_events_matrix, how_pick_events_vector

(<644x1332 sparse matrix of type '<class 'numpy.int64'>'
 	with 7899 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.4, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [53]:
lda14 = LatentDirichletAllocation(n_components= 2, random_state = 42)

In [54]:
lda14.fit(how_pick_events_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [55]:
pyLDAvis.sklearn.prepare(lda14, how_pick_events_matrix, how_pick_events_vector)

PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
1      0.09293  0.0       1        1  53.676519
0     -0.09293  0.0       2        1  46.323481, topic_info=     Category        Freq             Term       Total  loglift  logprob
206   Default  194.000000       conference  194.000000  30.0000  30.0000
1155  Default  165.000000            topic  165.000000  29.0000  29.0000
262   Default  144.000000             cost  144.000000  28.0000  28.0000
71    Default  101.000000           attend  101.000000  27.0000  27.0000
1324  Default   51.000000             year   51.000000  26.0000  26.0000
684   Default   42.000000            local   42.000000  25.0000  25.0000
133   Default   57.000000           budget   57.000000  24.0000  24.0000
1158  Default   24.000000    topic covered   24.000000  23.0000  23.0000
990   Default   46.000000         research   46.000000  22.0000  22.0000
1190  Default  134.000000           travel  134.000000  21.0000  21.0000
292   Default   29.000000          covered   29.000000  20.0000  20.0000
105   Default   45.000000            based   45.000000  19.0000  19.0000
1151  Default   25.000000           timing   25.000000  18.0000  18.0000
328   Default   17.000000          depends   17.000000  17.0000  17.0000
911   Default   40.000000            price   40.000000  16.0000  16.0000
569   Default   17.000000               im   17.000000  15.0000  15.0000
784   Default   23.000000       networking   23.000000  14.0000  14.0000
973   Default   13.000000        relevance   13.000000  13.0000  13.0000
1014  Default   24.000000         schedule   24.000000  12.0000  12.0000
1073  Default   13.000000    speaker topic   13.000000  11.0000  11.0000
1218  Default   27.000000              try   27.000000  10.0000  10.0000
245   Default   20.000000    consideration   20.000000   9.0000   9.0000
239   Default   16.000000  conference year   16.000000   8.0000   8.0000
280   Default   12.000000      cost travel   12.000000   7.0000   7.0000
354   Default   12.000000         distance   12.000000   6.0000   6.0000
1281  Default   30.000000             want   30.000000   5.0000   5.0000
943   Default   18.000000          quality   18.000000   4.0000   4.0000
333   Default   14.000000           design   14.000000   3.0000   3.0000
322   Default   11.000000         decision   11.000000   2.0000   2.0000
1255  Default   16.000000               ux   16.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
575    Topic2    8.516176           impact    9.394761   0.6713  -6.1308
1155   Topic2  137.941355            topic  165.136236   0.5896  -3.3459
262    Topic2  114.035571             cost  144.778160   0.5308  -3.5362
245    Topic2   17.786645    consideration   20.518475   0.6266  -5.3943
105    Topic2   37.475565            based   45.358901   0.5786  -4.6491
133    Topic2   46.712350           budget   57.357862   0.5642  -4.4287
1014   Topic2   20.848439         schedule   24.810032   0.5956  -5.2355
943    Topic2   16.106512          quality   18.813790   0.6142  -5.4935
911    Topic2   31.379754            price   40.272511   0.5200  -4.8266
1190   Topic2   90.382494           travel  134.905817   0.3690  -3.7687
1133   Topic2   14.366558           ticket   17.110673   0.5947  -5.6079
1281   Topic2   23.724724             want   30.855178   0.5067  -5.1062
483    Topic2   11.143293              fit   12.823349   0.6291  -5.8619
460    Topic2   24.350180           factor   32.587509   0.4781  -5.0802
695    Topic2   64.105560         location  107.614991   0.2515  -4.1122
191    Topic2   31.981637          company   47.251638   0.3792  -4.8076
1301   Topic2   45.635564             work   80.119525   0.2067  -4.4521
1136   Topic2   47.541748             time   86.190099   0.1746  -4.4111
1064   Topic2   49.309648          speaker   93.138662   0.1336  -4.3746
632    Topic2   15.230718             know

#### Topic groups 

1. Local
2. Topic


In [56]:
lda_W14 = lda14.transform(how_pick_events_matrix)

In [57]:
top_doc_column14 = datadf.how_pick_events.dropna()

In [58]:
word_count_matrix14, count_vect14 = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [59]:
LDA14a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA14a.fit(word_count_matrix14)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [60]:
lda_H = LDA14a.transform(word_count_matrix14)

In [61]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column14, 3)

Top 3 Documents for Topic 0: 

Document 1
it is usually a combination of date and my availability to travel the location the size the speaker list and whether the event seems likely to cater to more senior researcher or designer i'm at a point where traveling far for 101 level talk is just not useful to me

Document 2
depends on the cost of the event eg is it an evening meetup v a conference if it's the latter then it would depend on company training budget or depending on how relevant the topic are for my personal development goal also depending on the talk itself eg what the topic are which company are presenting i am more interested in advanced topic around strategy prioritisation or maybe new method

Document 3
location is the primary factor to consider because it will indicate whether i can afford to attend or not my company will cover cost of entry to a conference or workshop but will not cover travel or accommodation i also use personal vacation time to attend learning session s

## 15. What was the best professional learning experience you've ever had?  What made it great?

In [62]:
datadf.best_event = datadf.best_event.dropna().apply(nlp.basic_clean)
datadf.best_event = datadf.best_event.dropna().apply(nlp.lemmatize)

In [63]:
best_event_matrix, best_event_vector = nlp.create_wordcount_matrix(datadf.best_event.dropna(), ngram=(1,3), max_df=.4)
best_event_matrix, best_event_vector

(<594x1631 sparse matrix of type '<class 'numpy.int64'>'
 	with 9198 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.4, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [64]:
lda15 = LatentDirichletAllocation(n_components= 2, random_state = 42)

In [65]:
lda15.fit(best_event_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [66]:
pyLDAvis.sklearn.prepare(lda15, best_event_matrix, best_event_vector)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.073775  0.0       1        1  64.038274
1     -0.073775  0.0       2        1  35.961726, topic_info=     Category        Freq          Term       Total  loglift  logprob
1438  Default   97.000000            ux   97.000000  30.0000  30.0000
1592  Default  134.000000      workshop  134.000000  29.0000  29.0000
48    Default   17.000000      activity   17.000000  28.0000  28.0000
118   Default   24.000000      attendee   24.000000  27.0000  27.0000
303   Default   36.000000        course   36.000000  26.0000  26.0000
559   Default   47.000000          good   47.000000  25.0000  25.0000
597   Default   31.000000         group   31.000000  24.0000  24.0000
774   Default   75.000000      learning   75.000000  23.0000  23.0000
1069  Default   11.000000       product   11.000000  22.0000  22.0000
829   Default   48.000000           lot   48.000000  21.0000  21.0000
966   Default   10.000000        online   10.000000  20.0000  20.0000
1229  Default   36.000000         small   36.000000  19.0000  19.0000
1627  Default    9.000000         young    9.000000  18.0000  18.0000
886   Default   10.000000        method   10.000000  17.0000  17.0000
376   Default   21.000000     different   21.000000  16.0000  16.0000
1472  Default    9.000000   ve attended    9.000000  15.0000  15.0000
350   Default   78.000000        design   78.000000  14.0000  14.0000
292   Default   10.000000  conversation   10.000000  13.0000  13.0000
1333  Default   76.000000          talk   76.000000  12.0000  12.0000
1113  Default   24.000000          real   24.000000  11.0000  11.0000
674   Default    8.000000          indi    8.000000  10.0000  10.0000
446   Default   74.000000         event   74.000000   9.0000   9.0000
196   Default    8.000000           chi    8.000000   8.0000   8.0000
1618  Default   47.000000          year   47.000000   7.0000   7.0000
1147  Default    8.000000      remember    8.000000   6.0000   6.0000
466   Default   74.000000    experience   74.000000   5.0000   5.0000
208   Default    8.000000         clear    8.000000   4.0000   4.0000
1007  Default   77.000000        people   77.000000   3.0000   3.0000
1393  Default   58.000000         topic   58.000000   2.0000   2.0000
248   Default  184.000000    conference  184.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
334    Topic2    8.223818  day workshop   10.061980   0.8210  -6.0707
597    Topic2   20.979208         group   31.257889   0.6240  -5.1342
376    Topic2   15.057778     different   21.948171   0.6459  -5.4659
1592   Topic2   61.704777      workshop  134.091507   0.2466  -4.0554
559    Topic2   27.353935          good   47.543798   0.4699  -4.8689
829    Topic2   26.490222           lot   48.464904   0.4187  -4.9010
1229   Topic2   20.793133         small   36.518085   0.4595  -5.1432
1350   Topic2    7.498494      teaching    9.225615   0.8154  -6.1631
1333   Topic2   33.808048          talk   76.951156   0.2002  -4.6571
446    Topic2   32.137698         event   74.415097   0.1831  -4.7077
1007   Topic2   32.694285        people   77.885597   0.1547  -4.6906
1393   Topic2   26.445717         topic   58.967824   0.2208  -4.9027
577    Topic2   41.157480         great  116.811313  -0.0204  -4.4604
611    Topic2   13.699140          hand   22.895671   0.5091  -5.5604
136    Topic2   20.127424          best   41.803860   0.2918  -5.1757
1483   Topic2   70.326382            wa  318.266333  -0.4870  -3.9246
1364   Topic2   16.717936         thing   31.485137   0.3897  -5.3613
1587   Topic2   20.646754          work   44.401301   0.2570  -5.1502
329    Topic2   24.492898           day   59.071965   0.1423  -4.9794
1152   Topic2   29.276769      research   85.943258  -0.0542  -4.8010
1511   Topic2   14.274138      wa great   26.365189   0.4091  -5.5193
1253   Topic2   23.677768       speaker   74.866254

#### Topic groups 

1. UX
2. Online


In [67]:
lda_W15 = lda15.transform(best_event_matrix)

In [68]:
top_doc_column15 = datadf.best_event.dropna()

In [69]:
word_count_matrix15, count_vect15 = nlp.create_wordcount_matrix(datadf.best_event.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [70]:
LDA15a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA15a.fit(word_count_matrix15)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [71]:
lda_H = LDA15a.transform(word_count_matrix15)

In [72]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column15, 3)

Top 3 Documents for Topic 0: 

Document 1
two polar opposite experience c2mtl highly produced inspiring talk specific networking and meeting discussion and opportunity in an adjacent area to my own not my core sme so not the same old people i used what i learned from that experience to design an incredible offsite for my organization a few month after i returned a 2day immersive workshopclass on analyzing data that i took from iit many year ago it wa full of tool some of which were new to me and handson feedback not only did i walk away with tool and skill that i didn't have before i used that experience to design immersive learning course in the future it wa one of the earliest learnbydoing class that i'd taken

Document 2
the best experience i had not ux focussed wa the impact factory training workshop that wa over 2 day called 'train the trainer' fantastic and engaging trainer and really interactive session with no slide deck at all i liked the ucd london conference in 2010 or 2011 

## 16. What if any events have you attended on the subject of research in the past few years?

In [73]:
datadf.events_attend_recent = datadf.events_attend_recent.astype('str')

In [74]:
datadf.events_attend_recent = datadf.events_attend_recent.dropna().apply(nlp.basic_clean)
datadf.events_attend_recent = datadf.events_attend_recent.dropna().apply(nlp.lemmatize)

In [75]:
events_attend_recent_matrix, events_attend_recent_vector = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), ngram=(1,3), max_df=.3)
events_attend_recent_matrix, events_attend_recent_vector

(<726x641 sparse matrix of type '<class 'numpy.int64'>'
 	with 3538 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [145]:
lda16 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [146]:
lda16.fit(events_attend_recent_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [147]:
pyLDAvis.sklearn.prepare(lda16, events_attend_recent_matrix, events_attend_recent_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.095808  0.152668       1        1  31.629233
1     -0.089995 -0.147376       2        1  28.617695
3     -0.016189 -0.011150       3        1  22.543404
0      0.201993  0.005858       4        1  17.209669, topic_info=    Category        Freq                       Term       Total  loglift  \
351  Default  140.000000                        nan  140.000000  30.0000   
599  Default   45.000000                       uxpa   45.000000  29.0000   
167  Default   41.000000                       epic   41.000000  28.0000   
294  Default   33.000000                     london   33.000000  27.0000   
561  Default   25.000000                       user   25.000000  26.0000   
567  Default  130.000000                         ux  130.000000  25.0000   
608  Default   16.000000                        uxr   16.000000  24.0000   
517  Default   22.000000                     strive   22.000000  23.0000   
136  Default   24.000000                        day   24.000000  22.0000   
562  Default   21.000000              user research   21.000000  21.0000   
428  Default  163.000000                   research  163.000000  20.0000   
102  Default  132.000000                 conference  132.000000  19.0000   
65   Default   12.000000                     boston   12.000000  18.0000   
437  Default   16.000000            research london   16.000000  17.0000   
550  Default   10.000000                    toronto   10.000000  16.0000   
563  Default   15.000000       user research london   15.000000  15.0000   
84   Default   18.000000                        chi   18.000000  14.0000   
408  Default   16.000000                       qrca   16.000000  13.0000   
601  Default    8.000000                uxpa boston    8.000000  12.0000   
195  Default   12.000000                    focused   12.000000  11.0000   
476  Default   18.000000             service design   18.000000  10.0000   
142  Default   58.000000                     design   58.000000   9.0000   
390  Default    9.000000                     people    9.000000   8.0000   
252  Default    7.000000              international    7.000000   7.0000   
376  Default   15.000000                     online   15.000000   6.0000   
176  Default   45.000000                      event   45.000000   5.0000   
475  Default   19.000000                    service   19.000000   4.0000   
518  Default    6.000000          strive conference    6.000000   3.0000   
604  Default    6.000000                 uxpa event    6.000000   2.0000   
282  Default   19.000000              local meetups   19.000000   1.0000   
..       ...         ...                        ...         ...      ...   
448   Topic4    2.736172            research summit    3.379825   1.5484   
419   Topic4    2.736172    radical research summit    3.379825   1.5484   
603   Topic4    2.736130            uxpa conference    3.379826   1.5484   
519   Topic4    2.735253  strive conference toronto    3.379845   1.5481   
513   Topic4    2.734948             steve portigal    3.379812   1.5480   
398   Topic4    2.734948                   portigal    3.379812   1.5480   
91    Topic4    2.734669                  chipchase    3.379858   1.5479   
263   Topic4    2.734669              jan chipchase    3.379858   1.5479   
306   Topic4    2.734511                masterclass    3.379861   1.5478   
612   Topic4    2.734008                uxr toronto    3.379802   1.5477   
118   Topic4    2.733519            conference uxpa    3.379796   1.5475   
291   Topic4    2.731161                 local uxpa    3.379769   1.5466   
404   Topic4    2.731052                 presenting    3.379879   1.5465   
119   Topic4    2.729517              conference ve    3.379843   1.5460   
517   Topic4   15.783372                     strive   22.020118   1.4267   
84    Topic4   10.128819                        chi   18.7448

#### Topic groups 

1. UX
2. UX
3. Epic
4. Uxpa



In [79]:
lda_W16 = lda16.transform(events_attend_recent_matrix)

In [80]:
top_doc_column16 = datadf.events_attend_recent.dropna()

In [81]:
word_count_matrix16, count_vect16 = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [82]:
LDA16a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA16a.fit(word_count_matrix16)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [83]:
lda_H = LDA16a.transform(word_count_matrix16)

In [84]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column16, 3)

Top 3 Documents for Topic 0: 

Document 1
uxr collective's strive conference in toronto

Document 2
nux manchester steve portigal's research networking dinner meetup at answerlab how to minimize research bias in your project

Document 3
strive local uxpa chapter meeting boston uxpa conference

Top 3 Documents for Topic 1: 

Document 1
researchops workshop user research sydney user research london ux insight utrecht

Document 2
research ops remote meetups ux monday in prague research tuesday in prague some ux conference meetups and barcamps in berlin bratislava brno prague and vienna

Document 3
not really research only but when i go to ux or web conference i pick research speaches nng exconf bratislava doer budapest webexpo prague ux salon tel aviv ux camp europe in berlin best thing ever mostly and i have a ticket for ux brighton

Top 3 Documents for Topic 2: 

Document 1
a 2 day ux design and research course numerous monthly nux event an nux conference in manchester

Document 2
actua

## 20. Did we miss any other types of conference sessions that you'd like to mention?

In [85]:
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.basic_clean)
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.lemmatize)

In [86]:
other_conference_types_matrix, other_conference_types_vector = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), ngram=(1,3), max_df=.3)
other_conference_types_matrix, other_conference_types_vector

(<258x295 sparse matrix of type '<class 'numpy.int64'>'
 	with 1136 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [87]:
lda20 = LatentDirichletAllocation(n_components= 2, random_state = 42)

In [88]:
lda20.fit(other_conference_types_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [89]:
pyLDAvis.sklearn.prepare(lda20, other_conference_types_matrix, other_conference_types_vector)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.105569  0.0       1        1  61.879519
1     -0.105569  0.0       2        1  38.120481, topic_info=    Category       Freq              Term      Total  loglift  logprob
216  Default  33.000000           session  33.000000  30.0000  30.0000
159  Default  15.000000              nope  15.000000  29.0000  29.0000
33   Default  31.000000        conference  31.000000  28.0000  28.0000
179  Default   8.000000            poster   8.000000  27.0000  27.0000
166  Default   7.000000             panel   7.000000  26.0000  26.0000
204  Default  18.000000          research  18.000000  25.0000  25.0000
22   Default   6.000000              case   6.000000  24.0000  24.0000
108  Default   6.000000         important   6.000000  23.0000  23.0000
246  Default  14.000000              talk  14.000000  22.0000  22.0000
284  Default  13.000000               way  13.000000  21.0000  21.0000
23   Default   5.000000        case study   5.000000  20.0000  20.0000
12   Default   6.000000          attendee   6.000000  19.0000  19.0000
180  Default   5.000000    poster session   5.000000  18.0000  18.0000
143  Default   4.000000             maybe   4.000000  17.0000  17.0000
241  Default   6.000000             study   6.000000  16.0000  16.0000
262  Default   8.000000             topic   8.000000  15.0000  15.0000
70   Default  14.000000             event  14.000000  14.0000  14.0000
167  Default   3.000000  panel discussion   3.000000  13.0000  13.0000
155  Default   5.000000        networking   5.000000  12.0000  12.0000
133  Default  18.000000              like  18.000000  11.0000  11.0000
101  Default   3.000000           helpful   3.000000  10.0000  10.0000
268  Default   5.000000      unconference   5.000000   9.0000   9.0000
52   Default   3.000000              demo   3.000000   8.0000   8.0000
288  Default   9.000000           working   9.000000   7.0000   7.0000
224  Default   3.000000             skill   3.000000   6.0000   6.0000
85   Default   3.000000              game   3.000000   5.0000   5.0000
168  Default   3.000000       participant   3.000000   4.0000   4.0000
236  Default   3.000000             speed   3.000000   3.0000   3.0000
149  Default   3.000000            method   3.000000   2.0000   2.0000
170  Default   3.000000             pecha   3.000000   1.0000   1.0000
..       ...        ...               ...        ...      ...      ...
285   Topic2   2.649938          webinars   3.084891   0.8124  -5.1847
211   Topic2   2.646704              role   3.085225   0.8111  -5.1859
43    Topic2   2.644849             cover   3.085418   0.8103  -5.1866
266   Topic2   2.634410          tutorial   3.086499   0.8060  -5.1906
163   Topic2   2.634228            option   3.086518   0.8060  -5.1907
99    Topic2   2.615808              hear   3.088426   0.7983  -5.1977
12    Topic2   5.774271          attendee   6.960487   0.7776  -4.4058
52    Topic2   3.176841              demo   3.870157   0.7670  -5.0034
212   Topic2   1.901360         role play   2.322586   0.7643  -5.5167
177   Topic2   1.901360              play   2.322586   0.7643  -5.5167
225   Topic2   1.901352        skill like   2.322587   0.7643  -5.5167
214   Topic2   2.531881               say   3.097119   0.7629  -5.2303
241   Topic2   4.959193             study   6.205071   0.7403  -4.5580
216   Topic2  25.429448           session  33.479271   0.6894  -2.9233
155   Topic2   4.242082        networking   5.439506   0.7158  -4.7142
204   Topic2  12.314739          research  18.040837   0.5826  -3.6485
262   Topic2   5.886488             topic   8.628555   0.5820  -4.3866
70    Topic2   8.168203             event  14.271125   0.4064  -4.0590
133   Topic2   9.636266              like  18.318286   0.3221  -3.8937
268   Topic2   3.963018      unconference   5.468413   0.6424  -4.7822
28    Topic2   3.413377              come   4.685502   0.6476  -4.931

#### Topic groups 

1. Industry
2. Case studies



In [90]:
lda_W20 = lda20.transform(other_conference_types_matrix)

In [91]:
top_doc_column20 = datadf.other_conference_types.dropna()

In [92]:
word_count_matrix20, count_vect20 = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [93]:
LDA20a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA20a.fit(word_count_matrix20)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [94]:
lda_H = LDA20a.transform(word_count_matrix20)

In [95]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column20, 3)

Top 3 Documents for Topic 0: 

Document 1
i am there to learn efficiently not socialize so the thing you didn't mention is the availability of course note at least the speakers' slide that i can study before and after the talk i have also been at conference that have a knowledgeable discussant associated with each 1hour talk the discussant ha studied the work before and asks pointed question about pro and con relationship to other technique point of failure etc when the speaker know this is coming you are le likely to get just a canned talk and you really get to the meat of the issue

Document 2
to make a conference really be incredible making something that they take away is critical cocreation is a good way to crystalize learning and having a physical artifact to take away and help you remember what you learned is great now that i wrote that i think it might be pretty cool to do a handson session on sketchnoting at the beginning of the conference so people who haven't ever done it be

## 21. Subjects you most want to see covered at a research conference

In [96]:
datadf.ideal_topics = datadf.ideal_topics.dropna().apply(nlp.basic_clean)
datadf.ideal_topics = datadf.ideal_topics.dropna().apply(nlp.lemmatize)

In [97]:
ideal_topics_matrix, ideal_topics_vector = nlp.create_wordcount_matrix(datadf.ideal_topics.dropna(), ngram=(1,3), max_df=.3)
ideal_topics_matrix, ideal_topics_vector

(<560x1122 sparse matrix of type '<class 'numpy.int64'>'
 	with 5854 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [121]:
lda21 = LatentDirichletAllocation(n_components= 2, random_state = 42)

In [122]:
lda21.fit(ideal_topics_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [123]:
pyLDAvis.sklearn.prepare(lda21, ideal_topics_matrix, ideal_topics_vector)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.100745  0.0       1        1  50.812903
1     -0.100745  0.0       2        1  49.187097, topic_info=     Category        Freq              Term       Total  loglift  logprob
582   Default  126.000000            method  126.000000  30.0000  30.0000
221   Default   46.000000              data   46.000000  29.0000  29.0000
132   Default   50.000000              case   50.000000  28.0000  28.0000
989   Default   57.000000             study   57.000000  27.0000  27.0000
134   Default   43.000000        case study   43.000000  26.0000  26.0000
1083  Default   40.000000                ux   40.000000  25.0000  25.0000
843   Default   23.000000   research method   23.000000  24.0000  24.0000
968   Default   22.000000       stakeholder   22.000000  23.0000  23.0000
560   Default   20.000000        management   20.000000  22.0000  22.0000
41    Default   46.000000          analysis   46.000000  21.0000  21.0000
351   Default   19.000000           finding   19.000000  20.0000  20.0000
712   Default   39.000000          practice   39.000000  19.0000  19.0000
887   Default   18.000000     research team   18.000000  18.0000  18.0000
1084  Default   20.000000       ux research   20.000000  17.0000  17.0000
37    Default   14.000000                ai   14.000000  16.0000  16.0000
58    Default   14.000000       application   14.000000  15.0000  15.0000
1012  Default   48.000000              team   48.000000  14.0000  14.0000
727   Default   16.000000           problem   16.000000  13.0000  13.0000
632   Default   14.000000        new method   14.000000  12.0000  12.0000
119   Default   12.000000          building   12.000000  11.0000  11.0000
764   Default   13.000000          question   13.000000  10.0000  10.0000
1018  Default   18.000000        technology   18.000000   9.0000   9.0000
244   Default   44.000000            design   44.000000   8.0000   8.0000
1041  Default   28.000000             topic   28.000000   7.0000   7.0000
99    Default   14.000000     best practice   14.000000   6.0000   6.0000
826   Default   11.000000  research finding   11.000000   5.0000   5.0000
1074  Default   30.000000              user   30.000000   4.0000   4.0000
600   Default   25.000000       methodology   25.000000   3.0000   3.0000
161   Default   11.000000         community   11.000000   2.0000   2.0000
1091  Default   15.000000             value   15.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
528    Topic2    6.270377              life    6.782016   0.6311  -6.1889
1023   Topic2    8.597886           testing    9.325223   0.6283  -5.8732
1084   Topic2   18.478087       ux research   20.345377   0.6133  -5.1081
1018   Topic2   16.573000        technology   18.649195   0.5915  -5.2169
777    Topic2   10.000383        recruiting   11.020395   0.6124  -5.7221
1091   Topic2   13.567835             value   15.258449   0.5921  -5.4170
1012   Topic2   36.788916              team   48.306007   0.4372  -4.4195
1074   Topic2   24.318519              user   30.511235   0.4827  -4.8335
244    Topic2   33.228504            design   44.067968   0.4272  -4.5213
755    Topic2   13.515519             quant   16.104520   0.5343  -5.4209
1053   Topic2    8.923073             trend   10.172054   0.5785  -5.8361
1078   Topic2   11.734583     user research   14.408587   0.5043  -5.5622
1107   Topic2   25.028411               way   38.128250   0.2886  -4.8047
1071   Topic2   12.632715               use   16.102745   0.4668  -5.4884
1016   Topic2   23.964456         technique   38.126111   0.2452  -4.8481
900    Topic2   22.727623        researcher   37.277449   0.2147  -4.9011
663    Topic2   18.009163      organization   27.960023   0.2696  -5.1338
654    Topic2   14.487603               ops   20.337356   0.3704  -5.3514
737    Topic2   13.589094           project   18.643197   0.3

#### Topic groups 

1. Method
2. Data



In [101]:
lda_W21 = lda21.transform(ideal_topics_matrix)

In [102]:
top_doc_column21 = datadf.ideal_topics.dropna()

In [103]:
word_count_matrix21, count_vect21 = nlp.create_wordcount_matrix(datadf.ideal_topics.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [104]:
LDA21a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA21a.fit(word_count_matrix21)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [105]:
lda_H = LDA21a.transform(word_count_matrix21)

In [106]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column21, 3)

Top 3 Documents for Topic 0: 

Document 1
researchops integrating research with design product and engineering growing and scaling a research practice leading a research team enterprise org structure for research team scaling the impact of research across an org research with emerging tech aimachine learning mrvrar multimodal experience etc

Document 2
facilitation skill conversation design master class teaching research basic to others skill for new research manager former ic moving into research management mixed method research design lightning improv shareout how to setup ux dashboard cxos qualinformed touchpoint specific outcome that matter most to people and ongoing quant measurement of those key experiencesmoments ethicalmoral framework for research that advance ai how do we consider downstream impact and just because we can doesnt mean we should

Document 3
i'd like to see if organization are still focusing on usability in my recent role there ha no longer been an emphasis on th

## 22. If attending a conference about research, who might you be excited to see there?

In [107]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [108]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.3)
ideal_att_matrix, ideal_att_vector

(<471x923 sparse matrix of type '<class 'numpy.int64'>'
 	with 4277 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [130]:
lda22 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [131]:
lda22.fit(ideal_att_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [132]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.099256  0.012285       1        1  26.978010
3      0.068893 -0.087438       2        1  25.507302
2     -0.020914  0.125629       3        1  25.495105
1     -0.147236 -0.050476       4        1  22.019582, topic_info=    Category       Freq              Term      Total  loglift  logprob
353  Default  40.000000          industry  40.000000  30.0000  30.0000
920  Default  16.000000             young  16.000000  29.0000  29.0000
514  Default  18.000000               new  18.000000  28.0000  28.0000
226  Default  14.000000             erika  14.000000  27.0000  27.0000
349  Default  12.000000        indi young  12.000000  26.0000  26.0000
348  Default  12.000000              indi  12.000000  25.0000  25.0000
227  Default  13.000000        erika hall  13.000000  24.0000  24.0000
264  Default  31.000000             field  31.000000  23.0000  23.0000
784  Default  22.000000             steve  22.000000  22.0000  22.0000
244  Default  22.000000        experience  22.000000  21.0000  21.0000
308  Default  15.000000              hall  15.000000  20.0000  20.0000
532  Default   8.000000            norman   8.000000  19.0000  19.0000
830  Default   8.000000             tomer   8.000000  18.0000  18.0000
406  Default  15.000000            ladner  15.000000  17.0000  17.0000
705  Default  15.000000               sam  15.000000  16.0000  16.0000
706  Default  15.000000        sam ladner  15.000000  15.0000  15.0000
687  Default   9.000000     research team   9.000000  14.0000  14.0000
601  Default  18.000000          portigal  18.000000  13.0000  13.0000
464  Default  11.000000              love  11.000000  12.0000  12.0000
811  Default  20.000000              tech  20.000000  11.0000  11.0000
734  Default   7.000000            sharon   7.000000  10.0000  10.0000
786  Default  17.000000    steve portigal  17.000000   9.0000   9.0000
422  Default   8.000000      leader field   8.000000   8.0000   8.0000
419  Default  54.000000            leader  54.000000   7.0000   7.0000
445  Default  43.000000              like  43.000000   6.0000   6.0000
153  Default  11.000000              data  11.000000   5.0000   5.0000
831  Default   6.000000      tomer sharon   6.000000   4.0000   4.0000
520  Default   6.000000           nielsen   6.000000   3.0000   3.0000
544  Default  32.000000      organization  32.000000   2.0000   2.0000
650  Default   9.000000              real   9.000000   1.0000   1.0000
..       ...        ...               ...        ...      ...      ...
293   Topic4   2.637633  google microsoft   3.270208   1.2983  -5.9249
812   Topic4   5.911240      tech company   7.362999   1.2936  -5.1180
425   Topic4   5.088393           leading   6.543947   1.2617  -5.2678
509   Topic4   1.830903         nate bolt   2.456270   1.2194  -6.2900
718   Topic4   1.830903             scott   2.456270   1.2194  -6.2900
865   Topic4   5.861215     user research   8.188759   1.1788  -5.1265
784   Topic4  14.034420             steve  22.179972   1.0556  -4.2533
601   Topic4  10.760934          portigal  18.085203   0.9941  -4.5189
35    Topic4   4.163990    anthropologist   5.738417   1.1925  -5.4683
786   Topic4  10.012089    steve portigal  17.270355   0.9680  -4.5910
406   Topic4   8.592003            ladner  15.663440   0.9127  -4.7440
705   Topic4   8.592003               sam  15.663440   0.9127  -4.7440
706   Topic4   8.592003        sam ladner  15.663440   0.9127  -4.7440
773   Topic4   7.349209             spool  13.995644   0.8691  -4.9002
339   Topic4   4.294067              ideo   6.565608   1.0886  -5.4376
202   Topic4   8.297746               don  17.304115   0.7783  -4.7788
811   Topic4   8.716443              tech  20.675885   0.6495  -4.7296
112   Topic4  13.352545           company  49.728987   0.1984  -4.3031
547   Topic4   6.047930              orgs  11.536891   0.8674  -5.0951
783   Topic4

#### Topic groups 

1. Industry
2. Leader
3. company
4. Indi young



In [112]:
lda_W22 = lda22.transform(ideal_att_matrix)

In [113]:
top_doc_column22 = datadf.ideal_attendees.dropna()

In [114]:
word_count_matrix22, count_vect22 = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [115]:
LDA22a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA22a.fit(word_count_matrix22)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [116]:
lda_H = LDA22a.transform(word_count_matrix22)

In [117]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column22, 3)

Top 3 Documents for Topic 0: 

Document 1
people with experience in building company by applying research knowledge i think there are people out there speaking from a consulting view they donat take a many risk a the one iad like to learn from company are usually led by people with a techdesign background if research is creating a real value in company and if we believe research can lead the strategic vision of a company iall be excited to see people who have managed to achieve that

Document 2
i would like to hear from and network with other organizational leader i want to meet people i can learn from no pr or salestype presentation no product pushing although i'd like to learn about tool that people are using i am much more interested in strategy and how to manage operation for large team also speaking a someone on the consulting side it would be nice to hear from leader in that space a well a leader within organization some of the advice from design executive doesn't really apply to